In [2]:
import tensorflow as tf
import numpy as np

In [3]:
def read_data(path):
    filename = tf.matching_files(path)
    filename_queue = tf.train.string_input_producer(filename)
    reader = tf.TFRecordReader()
    _, example = reader.read(filename_queue)
    features = tf.parse_single_example(
        example,
        features={
            'image_raw': tf.FixedLenFeature([], tf.string),
            'label': tf.FixedLenFeature([], tf.int64),
        })

    images = tf.decode_raw(features['image_raw'], tf.uint8)
    images = tf.reshape(images, [1728])
    labels = tf.cast(features['label'], tf.int64)
    return images / 255, labels

In [4]:
def image_reshape(l):
    lab = np.zeros([len(l), 4, 11])
    for n in range(len(l)):
        for j, j_ in enumerate(str(l[n]) + ' ' * (4 - len(str(l[n])))):
            if j_ == ' ':
                lab[n][j][10] = 1
            else:
                lab[n][j][int(j_)] = 1
    return lab

In [5]:
def init_weights(shape):
    return tf.Variable(tf.random_normal(shape, stddev=0.1))


def init_bias(shape):
    return tf.Variable(tf.constant(0.1, shape=shape))


In [6]:
x_ = tf.placeholder(tf.float32, [None, 1728])
y = tf.placeholder(tf.float32, [None, 4, 11])
keep_prob = tf.placeholder(tf.float32, name='keep_prob')
x = tf.reshape(x_, [-1, 36, 48, 1], name='x')

# 第一层卷积
w1 = init_weights([3, 3, 1, 32])
b1 = init_bias([32])
conv1 = tf.nn.relu(tf.nn.conv2d(x, w1, strides=[1, 1, 1, 1], padding='SAME') + b1)  # shape=(?, 36, 48, 32)
# 池化
pool1 = tf.nn.max_pool(conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')  # # shape=(?, 18, 24, 32)

# 第二层卷积
w2 = init_weights([3, 3, 32, 64])
b2 = init_bias([64])
conv2 = tf.nn.relu(tf.nn.conv2d(pool1, w2, strides=[1, 1, 1, 1], padding='SAME') + b2)  # shape=(?, 18, 24, 64)
# 池化
pool2 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')  # shape=(?, 9, 12, 64)

# 第三层卷积
w3 = init_weights([3, 3, 64, 96])
b3 = init_bias([96])
conv3 = tf.nn.relu(tf.nn.conv2d(pool2, w3, strides=[1, 1, 1, 1], padding='SAME') + b3)  # shape=(?, 9, 12, 96)
# 池化
pool3_ = tf.nn.max_pool(conv3, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')  # shape=(?, 5, 6, 96)

pool3 = tf.reshape(pool3_, [-1, 5 * 6 * 96])

# 全连接层
w4 = init_weights([5 * 6 * 96, 512])
b4 = init_bias([512])
h = tf.nn.relu(tf.matmul(pool3, w4) + b4)
h = tf.nn.dropout(h, keep_prob)

# 输出层 1
w_o = init_weights([512, 11])
b_o = init_bias([11])
y_o = tf.matmul(h, w_o) + b_o


# 输出层 2
w_o1 = init_weights([512, 11])
b_o1 = init_bias([11])
y_o1 = tf.matmul(h, w_o1) + b_o1


# 输出层 3
w_o2 = init_weights([512, 11])
b_o2 = init_bias([11])
y_o2 = tf.matmul(h, w_o2) + b_o2

# 输出层 4
w_o3 = init_weights([512, 11])
b_o3 = init_bias([11])
y_o3 = tf.matmul(h, w_o3) + b_o3

y_oo = tf.concat([y_o, y_o1, y_o2, y_o3], 1)
y_oo = tf.reshape(y_oo, [-1, 4, 11])

cost1 = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_oo[:, 0], labels=y[:, 0]))
cost2 = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_oo[:, 1], labels=y[:, 1]))
cost3 = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_oo[:, 2], labels=y[:, 2]))
cost4 = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_oo[:, 3], labels=y[:, 3]))
# cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_oo, labels=y))
cost = tf.reduce_sum([cost1, cost2, cost3, cost4])
train_op = tf.train.AdamOptimizer().minimize(cost)
prediction = tf.equal(tf.argmax(y_oo, 2), tf.argmax(y, 2))
accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32), name='accuracy')

saver = tf.train.Saver()

batch_size = 100


In [7]:
with tf.Session() as sess:

    sess.run(tf.global_variables_initializer())
    image, label = read_data('./tfrecord/train*')
    images, labels = tf.train.shuffle_batch([image, label], batch_size=32000, num_threads=1,
                                            capacity=2000, min_after_dequeue=1000)
    v_img, v_lab = read_data('./tfrecord/ver*')
    v_image, v_label = tf.train.shuffle_batch([v_img, v_lab], batch_size=400, num_threads=2,
                                            capacity=2000, min_after_dequeue=1000)

    for i in range(100):

        coord = tf.train.Coordinator()
        thread = tf.train.start_queue_runners(coord=coord)
        img, ll = sess.run([images, labels])
        v_images, v_labels = sess.run([v_image, v_label])
        lab = image_reshape(ll)
        v_lab = image_reshape(v_labels)
        training_batch = zip(range(0, len(img), batch_size),
                             range(batch_size, len(img) + 1, batch_size))
        for start, end in training_batch:

            acc, co, a, yy, p = sess.run([accuracy, cost, train_op, y_oo, prediction],
                                                feed_dict={x_: img[start:end], y: lab[start:end], keep_prob: 1.0})

        print('训练次数：', i, '正确率：', acc, '代价：', co)
        if i % 5 == 0:
            v_accurary = sess.run(accuracy, feed_dict={x_: v_images, y: v_lab, keep_prob: 1.0})
            print('验证集：', v_accurary)

        if acc > 0.9 and v_accurary > 0.9:
            save_path = saver.save(sess, 'model.ckpt')
    coord.request_stop()
    coord.join(thread)

INFO:tensorflow:Error reported to Coordinator: <class 'tensorflow.python.framework.errors_impl.InvalidArgumentError'>, assertion failed: [string_input_producer requires a non-null input tensor]
	 [[Node: input_producer/Assert/Assert = Assert[T=[DT_STRING], summarize=3, _device="/job:localhost/replica:0/task:0/device:CPU:0"](input_producer/Greater, input_producer/Assert/Assert/data_0)]]


OutOfRangeError: RandomShuffleQueue '_1_shuffle_batch/random_shuffle_queue' is closed and has insufficient elements (requested 32000, current size 0)
	 [[Node: shuffle_batch = QueueDequeueManyV2[component_types=[DT_FLOAT, DT_INT64], timeout_ms=-1, _device="/job:localhost/replica:0/task:0/device:CPU:0"](shuffle_batch/random_shuffle_queue, shuffle_batch/n)]]

Caused by op 'shuffle_batch', defined at:
  File "/home/miles/.linuxbrew/Cellar/python3/3.6.3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/miles/.linuxbrew/Cellar/python3/3.6.3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/miles/.virtualenvs/env/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/miles/.virtualenvs/env/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/miles/.virtualenvs/env/lib/python3.6/